<a href="https://colab.research.google.com/github/bmil-ssu/advanced_ann/blob/main/3_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from sklearn.model_selection import train_test_split

from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np

In [13]:
# 중요! 해당 cell 이외의 cell은 수정하지 않습니다.

# 성능 평가를 위해 모델 구조를 동일하게 작성합니다.
ENCODER_1 = 128
ENCODER_2 = 63
ENCODER_3 = 12

LATENT_VECTOR = 3

DECODER_1 = 12
DECODER_2 = 64
DECODER_3 = 128

# AutoEncoder 모델을 정의합니다.
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_shape,ENCODER_1),
            nn.ReLU(),
            nn.Linear(ENCODER_1, ENCODER_2),
            nn.ReLU(),
            nn.Linear(ENCODER_2, ENCODER_3),
            nn.ReLU(),
            nn.Linear(ENCODER_3, LATENT_VECTOR)
        )

        self.decoder = nn.Sequential(
            nn.Linear(LATENT_VECTOR, DECODER_1),
            nn.ReLU(),
            nn.Linear(DECODER_1, DECODER_2),
            nn.ReLU(),
            nn.Linear(DECODER_2, DECODER_3),
            nn.ReLU(),
            nn.Linear(DECODER_3, input_shape),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [14]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = Autoencoder().to(device)
print(model)

Using cpu device
Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=63, bias=True)
    (3): ReLU()
    (4): Linear(in_features=63, out_features=12, bias=True)
    (5): ReLU()
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Sigmoid()
  )
)


In [15]:
# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [16]:
# 데이터로더를 생성합니다.
test_dataloader = DataLoader(test_data,batch_size=10000)

In [17]:
# 데이터의 형태를 확인합니다.
for X, y in test_dataloader:
    input_shape = X.shape[2] * X.shape[3]
    print(f"Shape of input: {input_shape}")
    num_class = len(y.unique())
    print(f"num of class: {num_class}")
    break

Shape of input: 784
num of class: 10


In [18]:
# test 함수를 정의합니다.
# 동등한 비교를 위해 모든 학생이 공통된 Loss function을 사용합니다. (학습때 사용한 function과 다를 수 있습니다.)
def test(dataloader, model):
    loss_fn = nn.MSELoss() # 임의로 수정하지 않습니다.
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch, (X,y) in enumerate(dataloader):
            X = X.view(-1,28*28).to(device)

            recon = model(X)
            loss = loss_fn(recon, X)
            test_loss += loss
    test_loss /= num_batches
    return test_loss

In [19]:
# 저장된 모델을 불러오기 위해 google drive를 연동합니다.
from google.colab import drive
drive.mount('/content/drive')

# 저장된 모델을 불러옵니다.
PATH = '/content/drive/MyDrive/autoencoder.pt'
model.load_state_dict(torch.load(PATH))
model.eval()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=63, bias=True)
    (3): ReLU()
    (4): Linear(in_features=63, out_features=12, bias=True)
    (5): ReLU()
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Sigmoid()
  )
)

In [20]:
# 최종 성능을 출력합니다.
test_loss = test(test_dataloader, model)
print(f"Test Error: {test_loss:>8f} \n")

Test Error: 0.028948 

